In [33]:
import numpy as np
import pandas as pd
import random
import pickle
import json

<h2>Convert rec files for Ranksys</h2>

In [22]:
# filename = f"/home/diego/chat-reranking/experiments/anime/itemid_to_name.pkl"
# with open(f"{filename}", 'rb') as fp:
#     itemid_to_name = pickle.load(fp)
# filename = f"/home/diego/chat-reranking/experiments/anime/itemname_to_id.pkl"
# with open(f"{filename}", 'rb') as fp:
#     itemname_to_id = pickle.load(fp)

In [23]:
# for i in [16498,31964,30276,20583,32182,33352,24833,34599,13601]:
#     print(f"{i}: {itemid_to_name[i]}")
# itemname_to_id["Assassination Classroom Second Season"]

In [4]:
recs_folder = "/home/diego/chat-reranking/experiments/anime/recs/reranked"
# recs_name = "pzt-fold_0-serendipity-chatgpt"
recs_name = "chatgpt0613-div-p1-pzt-fold_0_20_100"
top_n = 10

# read JSON file
f = open(f"{recs_folder}/{recs_name}.json")
data = json.load(f)
f.close()

In [5]:
train_folder = f"/home/diego/chat-reranking/experiments/anime/fold_0/train_val_data.csv"
training_data = pd.read_csv(f'{train_folder}', names=["userid", "itemid", "rating"], sep="\t")

In [9]:
# check whether some recommendations intersect with the training set
recs = {}
for entry in data:
    recs[entry['userid']] = entry['reranked_recs']
    user_train = list(training_data[training_data["userid"] == entry['userid']]["itemid"].values)
    for item in entry['reranked_recs']:
        if item in user_train:
            print(f"user {entry['userid']}: {item}")
            # print(item)
            # print(entry['recs'])
            # print("######")
# recs

user 286312: 31964
user 286312: 20583
user 286312: 24833
user 319750: 5958


In [24]:
# exclude users with less than 10 valid recommendations
none_users = 0
users_with_none = 0
total_users = len(recs)
with open(f"{recs_folder}/{recs_name}", "w") as f:
    for user in recs:
        if recs[user] is None:
            none_users += 1
            continue
        score = float(top_n)
        usr_str = ""
        for r in recs[user]:
            if r==-1:
                users_with_none += 1
                usr_str = None
                break
            usr_str += f"{user}\t{r}\t{score}\n"
            score -= 1.0
        if usr_str is not None:
            f.write(usr_str)

print(f"Total users: {total_users}")
print(f"None users: {none_users}")
print(f"Users with some None: {users_with_none}")

Total users: 30
None users: 0
Users with some None: 0


In [36]:
print("Tune the length of the candidate set: chatgpt")
max_pos = []
for entry in data:
    base = entry['recs']
    re_ranked = entry['reranked_recs']
    if len(re_ranked) == 0:
        continue
    
    def return_pos(i):
        if i in base:
            return base.index(i)
        else:
            return 0
        
    pos = [return_pos(r) for r in re_ranked]
    max_pos.append(max(pos))
    
print(f"Average max reranking pos: {np.mean(max_pos)}") 
print(f"Average std reranking pos: {np.std(max_pos)}")    

Tune the length of the candidate set: chatgpt
Average max reranking pos: 25.733333333333334
Average std reranking pos: 13.03567753854355


In [55]:
rerankers = ["MMR-pzt-fold_0_20_100", "RxQuAD-pzt-fold_0_20_100", "xQuAD-pzt-fold_0_20_100"]

print(f"Tune the length of the candidate set: baseline reranker {rerankers[0]}")
base_recs = pd.read_csv(f"/home/diego/chat-reranking/experiments/anime/recs/baselines/pzt-fold_0_20_100", 
                            names=["userid", "itemid", "rating"], sep="\t")
re_ranked = pd.read_csv(f"/home/diego/chat-reranking/experiments/anime/recs/reranked/{rerankers[2]}", 
                            names=["userid", "itemid", "rating"], sep="\t")

max_pos = []
for userid in re_ranked["userid"].unique()[:300]:
    base = list(base_recs[base_recs["userid"] == userid]["itemid"].values[:50])
    recs = re_ranked[re_ranked["userid"] == userid]["itemid"].values.tolist()
        
    pos = [base.index(r) for r in recs]
    max_pos.append(max(pos))
print(f"Average max reranking pos: {np.mean(max_pos)}") 
print(f"Average std reranking pos: {np.std(max_pos)}")  

Tune the length of the candidate set: baseline reranker MMR-pzt-fold_0_20_100
Average max reranking pos: 22.436666666666667
Average std reranking pos: 10.324049054943941


In [47]:
re_ranked.head(1)

,userid,itemid,rating
0,198183,27899,10.0


<h2>Results analysis</h2>

In [37]:
results_path = "/home/diego/chat-rerank/experiments/ml-100k/results/ranksys_eval"
baseline_recs = pd.read_csv(f"{results_path}/pzt-fold_0")
gpt_recs = pd.read_csv(f"{results_path}/pzt-fold_0-chatgpt.csv")
mmr_recs = pd.read_csv(f"{results_path}/MMR-pzt-fold_0.csv")
xquad_recs = pd.read_csv(f"{results_path}/xQuAD-pzt-fold_0.csv")
rxquad_recs = pd.read_csv(f"{results_path}/RxQuAD-pzt-fold_0.csv")
gpt_seren_recs = pd.read_csv(f"{results_path}/pzt-fold_0-serendipity-chatgpt.csv")

In [38]:
d = {"reranker": ["baseline (MF)", "chat-gpt", "MMR", "xQuAD", "RxQuAD", "chat-gpt-S"]}
print("ChatGPT better than baseline in the following metrics:")
for col in gpt_recs.columns:
    d[col] = [baseline_recs[col][0], gpt_recs[col][0], mmr_recs[col][0], xquad_recs[col][0], rxquad_recs[col][0], gpt_seren_recs[col][0]]
    if gpt_recs[col][0] > baseline_recs[col][0]:
        print(col)
results = pd.DataFrame(data=d)

ChatGPT better than baseline in the following metrics:
s_recall_norel
eild_norel
binomial_norel
cbs_norel
cbs_rel
epc_norel
efd_norel
epd_norel
epd_rel


In [39]:
results

,reranker,prec,recall,ndcg,s_recall_norel,s_recall_rel,a_ndcg,err_ia,eild_norel,eild_rel,binomial_norel,binomial_rel,cbs_norel,cbs_rel,epc_norel,epc_rel,efd_norel,efd_rel,epd_norel,epd_rel
0,baseline (MF),0.227147,0.267024,0.290306,0.501926,0.206117,0.322673,0.155372,0.779424,0.158282,0.396710,0.311174,0.150426,0.022305,0.795201,0.175301,8.874204,1.976686,0.787220,0.181073
1,chat-gpt,0.163203,0.165964,0.207276,0.582519,0.171792,0.250379,0.123352,0.832728,0.112242,0.437842,0.282118,0.196504,0.022438,0.800794,0.125148,8.956339,1.415414,0.824287,0.824287
2,MMR,0.202863,0.237428,0.265213,0.647821,0.208015,0.308786,0.147390,0.908503,0.152934,0.630131,0.342383,NaN,NaN,0.802352,0.156628,8.963094,1.767976,0.845099,0.845099
3,xQuAD,0.219512,0.252236,0.271010,0.599263,0.232293,0.358666,0.173823,0.801650,0.153373,0.349369,0.306487,NaN,NaN,0.791737,0.167227,8.860434,1.895577,0.803579,0.803579
4,RxQuAD,0.224178,0.258343,0.277610,0.554948,0.228107,0.355004,0.180848,0.798137,0.159741,0.359947,0.318495,NaN,NaN,0.791228,0.171029,8.848852,1.937397,0.793750,0.793750
5,chat-gpt-S,0.200848,0.231910,0.253708,0.497293,0.183848,0.285446,0.135465,0.782174,0.136243,0.382823,0.291398,0.145918,0.019126,0.787908,0.155917,8.788886,1.754334,0.790241,0.790241


In [66]:
filename = f"/home/diego/chat-rerank/experiments/ml-100k/itemid_to_name.pkl"
with open(f"{filename}", 'rb') as fp:
    itemid_to_name = pickle.load(fp)

In [67]:
itemid_to_name[274]

'Sabrina'

In [68]:
filename = f"/home/diego/chat-rerank/experiments/ml-100k/itemname_to_id.pkl"
with open(f"{filename}", 'rb') as fp:
    itemname_to_id = pickle.load(fp)

In [69]:
itemname_to_id["Sabrina"]

486

In [56]:
filename = f"/home/diego/chat-rerank/experiments/ml-100k/df_items.csv"
df_items = pd.read_csv(filename)
df_items.head(1)

,movietitle,name,genres
0,Toy Story (1995),Toy Story,"['Animation', 'Childrens', 'Comedy']"


In [58]:
df_items.duplicated().value_counts()

False    1664
True       18
dtype: int64